In [1]:
import numpy as np
import pandas as pd

In [2]:

# Read the saved dataset
city_data_df = pd.read_csv('city_ahp.csv', index_col=0)

# Convert DataFrame into dictionary
city_data = city_data_df.transpose().to_dict(orient='list')

cities = ['Toronto', 'Vancouver', 'Montreal', 'Ottawa', 'Calgary']

main_criteria = ['Population', 'Income', 'Transportation']
sub_criteria = {
    'Population': ['Population Density', 'Population Growth Rate', 'Quality of Life'],
    'Income': ['Average Income', 'Employment Opportunities', 'Cost of Living'],
    'Transportation': ['Public Transport', 'Traffic Congestion', 'Accessibility']
}

# Pairwise comparison matrices for main criteria and sub-criteria
main_criteria_matrix = np.array([
    [1, 5, 3],
    [1/5, 1, 1/3],
    [1/3, 3, 1]
])

sub_criteria_matrices = {
    'Population Density': np.array([
        [1, 3, 5],
        [1/3, 1, 3],
        [1/5, 1/3, 1]
    ]),
    'Population Growth Rate': np.array([
        [1, 1/7, 1/3],
        [7, 1, 5],
        [3, 1/5, 1]
    ]),
    'Quality of Life': np.array([
        [1, 5, 3],
        [1/5, 1, 1/3],
        [1/3, 3, 1]
    ]),
    'Average Income': np.array([
        [1, 5, 3],
        [1/5, 1, 1/3],
        [1/3, 3, 1]
    ]),
    'Employment Opportunities': np.array([
        [1, 3, 5],
        [1/3, 1, 3],
        [1/5, 1/3, 1]
    ]),
    'Cost of Living': np.array([
        [1, 1/5, 1/3],
        [5, 1, 3],
        [3, 1/3, 1]
    ]),
    'Public Transport': np.array([
        [1, 3, 5],
        [1/3, 1, 3],
        [1/5, 1/3, 1]
    ]),
    'Traffic Congestion': np.array([
        [1, 1/7, 1/3],
        [7, 1, 5],
        [3, 1/5, 1]
    ]),
    'Accessibility': np.array([
        [1, 5, 3],
        [1/5, 1, 1/3],
        [1/3, 3, 1]
    ]),
}

# AHP function to calculate weights for main criteria and sub-criteria
def ahp(matrix):
    n = matrix.shape[0]
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_eigenvalue_index = np.argmax(eigenvalues)
    max_eigenvalue = eigenvalues[max_eigenvalue_index]
    weights = eigenvectors[:, max_eigenvalue_index].real
    weights /= np.sum(weights)
    return weights

# Calculate weights for main criteria
main_criteria_weights = ahp(main_criteria_matrix)

# Calculate weights for main criteria
main_criteria_weights = ahp(main_criteria_matrix)

# Calculate weights for sub-criteria
sub_criteria_weights = {}
for main_criterion, sub_criterion_list in sub_criteria.items():
    sub_weights = []
    for sub_criterion in sub_criterion_list:
        sub_weights.append(ahp(sub_criteria_matrices[sub_criterion]))
    sub_criteria_weights[main_criterion] = np.mean(sub_weights, axis=0)

# Calculate the final weights for all criteria (main criteria and sub-criteria combined)
all_criteria = main_criteria + [sub_criterion for sub_criteria_list in sub_criteria.values() for sub_criterion in sub_criteria_list]
all_criteria_weights = np.zeros(len(all_criteria))

for i, criterion in enumerate(all_criteria):
    if criterion in main_criteria:
        all_criteria_weights[i] = main_criteria_weights[main_criteria.index(criterion)]
    else:
        for sub_criterion_list in sub_criteria.values():
            if criterion in sub_criterion_list:
                main_criterion = list(sub_criteria.keys())[list(sub_criteria.values()).index(sub_criterion_list)]
                all_criteria_weights[i] = sub_criteria_weights[main_criterion][sub_criterion_list.index(criterion)]
                break

# Initialize an empty dictionary to store the scores for each city
city_scores = {}

# For each city, calculate the score as the sum of criteria value times its weight
for city, criteria_values in city_data.items():
    city_scores[city] = sum([value * weight for value, weight in zip(criteria_values, all_criteria_weights)])

# Find the city with the highest overall score
best_city = max(city_scores, key=city_scores.get)

# Print the scores for each city
print("City scores:")
for city, score in city_scores.items():
    print(f"{city}: {score}")

# Print the city with the highest score
print(f"\nBest city for expansion is: {best_city}")

City scores:
Toronto: 1902766.7360039828
Vancouver: 452098.04364944703
Montreal: 1149482.3870354744
Ottawa: 665062.1039554093
Calgary: 884708.3483721282

Best city for expansion is: Toronto
